In [12]:
# Make a new folder for random forest preiction for rental price
import os
  
# Directory
directory = "random_forest_pred"
  
# Parent Directory path
parent_dir = "../data/curated/"

# Path
path = os.path.join(parent_dir, directory)

# Create the directory
os.mkdir(path)

In [13]:
import pandas as pd
import glob
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import numpy as np

# Predict for the next 5 years
import os
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
parent_dir = "../data/curated/merged_dataset"

sel_pred = RandomForestRegressor(n_estimators = 100)

for filename in os.listdir(parent_dir):
    print(filename, "----------------------------------------------------------------------------------------------------------------")
    merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

    merged_df_yr = merged_df_yr.drop("address","latitude","longitude","postcode","sa2_2016")
    
    for c in merged_df_yr.columns:
        if (c not in  ['address', 'residence_type']):
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 

    merged_df_yr = merged_df_yr.toPandas()

    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes

    merged_df_yr.iloc[:, 13:21] = merged_df_yr.iloc[:, 13:21].replace(np.nan, 99999)

    merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
    
    merged_df_yr = merged_df_yr.dropna()

    
    X = merged_df_yr.drop(['weekly_rent'], axis=1)
    y = np.log(merged_df_yr['weekly_rent'])
    # Train with the whole dataset for the actual prediction for the next 5 years (This training is not for feature enginnering & accuracy test)
    sel_pred.fit(X, y)

2013_merged_data.csv ----------------------------------------------------------------------------------------------------------------


2021_merged_data.csv ----------------------------------------------------------------------------------------------------------------


2020_merged_data.csv ----------------------------------------------------------------------------------------------------------------


2022_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2018_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2014_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2017_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2015_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2019_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2016_merged_data.csv ----------------------------------------------------------------------------------------------------------------


In [14]:
# Predict for the next 5 years
import os
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

parent_dir = "../data/curated/2023_2027_data"

for filename in os.listdir(parent_dir):
    merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

    # Extract year from the file name 
    which_year = re.findall(r'\d+', filename)

    # Add year column to the dataset to fit the input into the model
    merged_df_yr = merged_df_yr.withColumn("year", lit(which_year[0]))

    for c in merged_df_yr.columns:
        merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 

    merged_df_yr = merged_df_yr.toPandas()
    merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes
    merged_df_yr.dropna(inplace=True)

    # Reorder the columns 
    merged_df_yr_reordered = merged_df_yr[['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar',
       'min_distance_to_cbd', 'min_distance_to_park', 'min_distance_to_prim',
       'min_distance_to_second', 'min_distance_to_train',
       'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop',
       'gdp', 'saving_rate', 'income_per_person', 'population_density',
       'crime_cases']]

    # Predict with random forest tree
    prediction = sel_pred.predict(merged_df_yr_reordered)

    new_csv_name = "../data/curated/random_forest_pred/" + filename

    np.savetxt(new_csv_name, prediction, delimiter=",")